In [2]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import gzip
import torch

/home/peter8780/miniconda3/envs/oneformer/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
m2f_data_dir = Path("../data/m2f_outputs").resolve()
assert m2f_data_dir.exists()

onef_data_dir = Path("../data/onef_outputs").resolve()
assert onef_data_dir.exists()

In [4]:
m2f_data_path = m2f_data_dir / "1.ptz"
assert m2f_data_path.exists()

onef_data_path = onef_data_dir / "oneformer.ptz"
assert onef_data_path.exists()

In [5]:
with gzip.open(m2f_data_path, "r") as f:
    m2f_data = torch.load(f)
print(m2f_data.keys())

with gzip.open(onef_data_path, "r") as f:
    onef_data = torch.load(f)
[onef_data] = onef_data

dict_keys(['mask', 'segments', 'mask_notta', 'segments_notta', 'confidences_notta', 'probabilities', 'confidences'])


In [6]:
# mask
print(m2f_data["mask"])
print(onef_data["segmentation"])
converted_mask = onef_data["segmentation"].to("cuda")
# print(converted_mask)

tensor([[0, 0, 0,  ..., 7, 7, 7],
        [0, 0, 0,  ..., 7, 7, 7],
        [0, 0, 0,  ..., 7, 7, 7],
        ...,
        [6, 6, 6,  ..., 6, 6, 6],
        [6, 6, 6,  ..., 6, 6, 6],
        [6, 6, 6,  ..., 6, 6, 6]], device='cuda:0', dtype=torch.int32)
tensor([[2, 2, 2,  ..., 2, 2, 2],
        [2, 2, 2,  ..., 2, 2, 2],
        [2, 2, 2,  ..., 2, 2, 2],
        ...,
        [5, 5, 5,  ..., 3, 3, 3],
        [5, 5, 5,  ..., 3, 3, 3],
        [5, 5, 5,  ..., 3, 3, 3]], dtype=torch.int32)


In [7]:
# segments
# "isthhing" is not "was_fused", so panoptic lift should not use it.
print(m2f_data["segments"])
print(onef_data["segments_info"])

converted_segments = []
for d in onef_data["segments_info"]:
    new_dict = {'id': d['id'], 'isthing': d['was_fused'], 'category_id': d['label_id'], 'score': d['score']}
    converted_segments.append(new_dict)
# print(converted_segments)



[{'id': 1, 'isthing': False, 'category_id': 21, 'score': 0.9976209402084351}, {'id': 2, 'isthing': False, 'category_id': 7, 'score': 0.9981362223625183}, {'id': 3, 'isthing': True, 'category_id': 5, 'score': 0.9849984645843506}, {'id': 4, 'isthing': True, 'category_id': 5, 'score': 0.8396998047828674}, {'id': 5, 'isthing': False, 'category_id': 19, 'score': 0.9967018961906433}, {'id': 6, 'isthing': False, 'category_id': 2, 'score': 0.9982190132141113}, {'id': 7, 'isthing': False, 'category_id': 1, 'score': 0.9987754225730896}, {'id': 8, 'isthing': True, 'category_id': 5, 'score': 0.9743933081626892}, {'id': 9, 'isthing': True, 'category_id': 16, 'score': 0.9995818734169006}, {'id': 10, 'isthing': True, 'category_id': 20, 'score': 0.9905605912208557}]
[{'id': 1, 'label_id': 110, 'was_fused': False, 'score': 0.963935}, {'id': 2, 'label_id': 119, 'was_fused': False, 'score': 0.999676}, {'id': 3, 'label_id': 100, 'was_fused': False, 'score': 0.969604}, {'id': 4, 'label_id': 116, 'was_fused

In [23]:
# probabilities
print(m2f_data["probabilities"].shape)
print(onef_data["probability_map"].shape)
converted_probabilites = onef_data["probability_map"].to("cuda")
# print(converted_probabilites)


torch.Size([968, 1296, 22])
torch.Size([512, 683, 134])


In [29]:
# 3. Max Probability: maximum probability over all classes
binary_mask = m2f_data["mask"]==7
print(m2f_data["confidences_notta"])
max_prob = torch.amax(m2f_data["probabilities"], dim=-1)
print("max_prob")
print(max_prob)


print("origin")
print(m2f_data["confidences"])
print(onef_data.keys())

tensor([[1.0000, 1.0000, 1.0000,  ..., 0.9508, 0.8944, 0.5970],
        [1.0000, 1.0000, 1.0000,  ..., 0.9508, 0.8944, 0.5970],
        [1.0000, 1.0000, 1.0000,  ..., 0.9565, 0.9069, 0.6288],
        ...,
        [0.9901, 0.9901, 0.9910,  ..., 0.9669, 0.9354, 0.7445],
        [0.9890, 0.9890, 0.9900,  ..., 0.9615, 0.9252, 0.7186],
        [0.9890, 0.9890, 0.9900,  ..., 0.9615, 0.9252, 0.7186]],
       device='cuda:0')
max_prob
tensor([[1.0000, 1.0000, 1.0000,  ..., 0.9988, 0.9988, 0.9988],
        [1.0000, 1.0000, 1.0000,  ..., 0.9988, 0.9988, 0.9988],
        [1.0000, 1.0000, 1.0000,  ..., 0.9988, 0.9988, 0.9988],
        ...,
        [0.9982, 0.9982, 0.9982,  ..., 0.9982, 0.9982, 0.9982],
        [0.9982, 0.9982, 0.9982,  ..., 0.9982, 0.9982, 0.9982],
        [0.9982, 0.9982, 0.9982,  ..., 0.9982, 0.9982, 0.9982]],
       device='cuda:0')
origin
tensor([[1.0000, 1.0000, 1.0000,  ..., 0.9436, 0.8813, 0.5912],
        [1.0000, 1.0000, 1.0000,  ..., 0.9436, 0.8813, 0.5912],
        [1.0

In [10]:
converted_onef_data = {key:None for key in m2f_data.keys()}
converted_onef_data["mask"] = converted_mask
converted_onef_data["segments"] = converted_segments
converted_onef_data["probabilities"] = converted_probabilites